In [3]:
#Importing all dependencies
import pandas as pd
import numpy as np 
import os
import math
import itertools
import json
import time
import datetime
import requests
from datetime import date, timedelta, datetime, time
from calendar import monthrange

In [101]:
#Ignore for now

#https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/2021-03-24.csv

In [104]:
#Ignore for now

#df_shortsqueeze = pd.read_csv('https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/2021-03-24.csv')
#df_shortsqueeze

In [ ]:
#S&P 500 Analysis using Quicksight
#The S&P 500 Index contains 500 stocks/tickers/companies/symbols 
# Each day we process daily and minute data of these symbols

In [ ]:
# Raw Data comes in the form of daily data and minute e.g. 20210701_d.csv and 20210701_m.csv 
# Whereby the file name gives the data and d for daily and m for minute
# Each csv contains initial data that comprises mainly Open, High, Low, Close Volume and other
# Raw Daily Data comes from TDAmeritrade API 
# Raw Minute Data comes from Quandl API
# Other data will come from financialmodelingprep, shortsqueeze, bloomberg 
# financialmodelingprep api provides all the profile data and others
# shortsqueeze provides proprietary additional data calculated by shortsqueeze 
# bloomberg provides missing data we could not obtain from the above sources

In [44]:
# An example of Profile data, this will makes sense in a little while

url = "https://financialmodelingprep.com/api/v3/profile/MMM?apikey=aa8b4631b2ad1b2704741d1a2d6a2611"

response = requests.get(url)
data = response.json()
df_MMM_profile_all_data = pd.DataFrame(data)
df_MMM_profile = df_MMM_profile_all_data[["symbol","companyName","cik","industry","sector","exchangeShortName"]]
df_MMM_profile

#Note the cik number is unique to the company, even if the symbol changes or the name the cik will stay

,symbol,companyName,cik,industry,sector,exchangeShortName
0,MMM,3M Company,0000066740,Conglomerates,Industrials,NYSE


In [14]:
# While preparing teh files for this Code Snippet I accidental names daily - m and minute - d 
# However I will apply the correct letter when naming the dataframe

# Example of Raw daily data
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210625_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210628_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210629_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210630_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210701_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210702_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210706_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210707_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210708_d.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210709_d.csv

#Dataframe of 20210625_m.csv == df_20210625_d
df_20210625_d = pd.read_csv('https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210625_m.csv')
df_20210625_d
    

,Ticker,open,high,low,close,volume
0,A,147.01,147.760,146.21,147.28,8445127
1,AA,36.95,37.110,35.78,35.80,32496825
2,AAL,22.33,22.480,22.07,22.22,18738629
3,AAP,202.17,205.460,202.00,204.43,675038
4,AAPL,133.46,133.890,132.81,133.11,70783746
...,...,...,...,...,...,...
598,YUM,116.70,117.660,116.52,117.25,5229898
599,ZBH,164.53,167.590,164.53,165.85,1072442
600,ZBRA,513.17,514.595,506.47,510.65,251391
601,ZION,54.24,55.840,53.98,55.48,2221351


In [15]:
# Example of Raw daily data
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210625_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210628_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210629_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210630_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210701_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210702_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210706_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210707_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210708_m.csv
# https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210709_m.csv

# Dataframe of 20210625_d.csv == df_20210625_m
# This is for one day
df_20210625_m = pd.read_csv('https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/20210625_d.csv')
df_20210625_m

,Date,Timestamp,Ticker,OpenPrice,HighPrice,LowPrice,ClosePrice,TotalVolume,TotalQuantity,TotalTradeCount
0,20210625,04:28:00,MS,87.53,87.53,87.53,87.53,1.750600e+04,200,2
1,20210625,04:43:00,MS,87.54,87.54,87.54,87.54,8.754000e+03,100,1
2,20210625,06:40:00,MS,87.60,87.60,87.60,87.60,9.636000e+04,1100,2
3,20210625,07:00:00,MS,87.49,87.50,87.49,87.50,5.092400e+04,582,3
4,20210625,07:01:00,MS,87.68,87.76,87.68,87.76,6.036128e+04,688,3
...,...,...,...,...,...,...,...,...,...,...
180465,20210625,16:04:00,BLK,874.03,874.03,874.03,874.03,8.431767e+06,9647,3
180466,20210625,16:05:00,BLK,874.03,874.03,874.03,874.03,9.614330e+05,1100,3
180467,20210625,16:11:00,BLK,874.03,874.03,874.03,874.03,4.676060e+05,535,1
180468,20210625,16:41:00,BLK,874.03,874.03,874.03,874.03,1.076272e+08,123139,1


In [16]:
# The data is then extracted by symbol and columns renamed keeping all that we need.
# A lot of cleaning is done at this stage which involves:
# a.) Applying Name symbols when a company is bought or changes a name
# e.g. Recently Facebook with the symbol FB changed to Meta with the symbol META.
# This means all previous data FB data needs to be re-symboled from FB to META
# b.) Applying stock spilts - A company can announce a stock spilt
# A stock split is when the price of a company is divided, lets say Facebook FB has a 2 to 1 split applied tomorrow
# This means that if the price of Facebook was 100 and volume traded was 15000...
#... Tomorrow the 2 to 1 split means Facebook's price is 50 (halved) but the volume is 30000 (doubled)
# As a result all previous data needs to be adjusted.
# c.) The S&P 500 is a strict index sometimes a company is kicked out and replaced.
# This is why our data may have more than 500 stocks, financialmodelingprep api provides the correct list daily.
# Stock spilts and Name changes rarely happen but we have a script that checks for a, b and c situations.


In [19]:
# We will use MMM in the example below.
# MMM is the Symbol for a company listed in the S&P 500, the formal name is called 3M Co
#MMM_ALLDATADAILY.csv https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/ALLDAILYDATA.csv

# This function is an example of some cleaning. 
# It takes raw data contain many stocks and allows us to extract the symbol we want.
def get_all_data_for_symobol_daily(symbol):
    """Gets all  daily data for a given ticker
    
    Arguments : 
    symbol : what symbol to get data for 
    """
    big_df = pd.read_csv("https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/ALLDAILYDATA.csv")
    big_df = big_df[big_df['Ticker'] == symbol]  # filtering
    big_df = big_df.dropna(how = 'all',subset = ['Date',"Ticker"])
    big_df.to_csv(f"{symbol}_ALLDATADAILY.csv")



In [29]:
# Getting MMM daily data
get_all_data_for_symobol_daily("MMM")

#Dataframe of MMM_ALLDATADAILY.csv == df_MMM_d after applying get_data_presentation_type_1 function
df_MMM_d = pd.read_csv('MMM_ALLDATADAILY.csv')
df_MMM_d

,Unnamed: 0,Unnamed: 0.1,Ticker,open,high,low,close,volume,Date
0,384,384,MMM,86.410,86.900,85.83,86.72,2396791,20101222
1,1007,374,MMM,70.870,70.990,69.39,69.63,4832600,20080728
2,1635,381,MMM,79.880,80.590,78.93,79.02,4022482,20111031
3,2263,379,MMM,178.120,178.850,177.85,178.82,1226260,20160810
4,2846,332,MMM,53.275,54.445,53.01,54.35,3485800,20011025
...,...,...,...,...,...,...,...,...,...
5945,3562502,379,MMM,178.370,178.900,177.94,178.39,1098851,20160809
5946,3563128,375,MMM,70.840,71.250,70.29,70.39,4254180,20080731
5947,3563756,381,MMM,81.530,81.840,80.50,81.00,5252021,20111028
5948,3564389,385,MMM,139.500,141.490,139.12,140.81,3997985,20140501


In [30]:
#Dataframe of MMM_ALLDATADAILY.csv == df_MMM_m after applying get_data_presentation_type_1 function
df_MMM_m = pd.read_csv('https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/MMM_ALLDATAMINUTE.csv')
df_MMM_m

,Unnamed: 0,Unnamed: 0.1,Date,Timestamp,Ticker,OpenPrice,HighPrice,LowPrice,ClosePrice,TotalVolume,TotalQuantity,TotalTradeCount
0,47725,47725,20210709,08:31:00,MMM,198.98,198.98,198.98,198.98,19898.00,100,1
1,47726,47726,20210709,09:31:00,MMM,199.25,199.90,199.25,199.82,1400854.69,7015,48
2,47727,47727,20210709,09:32:00,MMM,199.51,199.90,199.35,199.69,829554.45,4156,34
3,47728,47728,20210709,09:33:00,MMM,199.84,199.90,199.56,199.89,1652102.89,8274,20
4,47729,47729,20210709,09:34:00,MMM,199.91,200.17,199.75,200.16,1580288.29,7901,37
...,...,...,...,...,...,...,...,...,...,...,...,...
25173,11443755,86793,20210412,16:39:00,MMM,197.47,197.50,197.47,197.50,122246.50,619,6
25174,11443756,86794,20210412,16:40:00,MMM,197.65,197.65,197.65,197.65,79060.00,400,2
25175,11443757,86795,20210412,16:55:00,MMM,197.65,197.65,197.65,197.65,19765.00,100,1
25176,11443758,86796,20210412,18:07:00,MMM,197.65,197.65,197.65,197.65,73921.10,374,2


In [31]:
# Because the data is ordered from oldest to newest we call this Type_1 data. newest to oldest would be Type_2 data
#Type_1 data is the format we will use to apply all the indicators e.g. Moving averages.

def get_data_presentation_type_1(data,ascending = True):
	"""Convert the DataFrame into type1(order by Symbol then Date then Timestamp)
	If ascending is passed as False automatically converts to type 2 instead 

	Args:
	data :The data to convert"""
	if "Timestamp" in data.columns : 
			data["Time"] = data['Timestamp']
			del data['Timestamp']
			data['Symbol'] = data['Ticker']
			del data['Ticker']
			data['Symbol'] = data.Symbol.astype(str).str.upper()
			data['Open'] = data['OpenPrice']
			del data['OpenPrice']
			data['Close'] = data['ClosePrice']
			del data['ClosePrice']
			data['High'] = data['HighPrice']
			del data['HighPrice']
			data['Low'] = data['LowPrice']
			del data['LowPrice']
			data['Volume'] = data['TotalVolume']
			del data['TotalVolume']
			data = data.sort_values(by = ['Symbol',"Date","Time"], ascending= ascending)

	else : 
			data['Symbol'] = data['Ticker']
			del data['Ticker']
			data['Symbol'] = data.Symbol.astype(str).str.upper()
			data['Open'] = data['open']
			del data['open']
			data['Close'] = data['close']
			del data['close']
			data['High'] = data['high']
			del data['high']
			data['Low'] = data['low']
			del data['low']
			data['Volume'] = data['volume']
			del data['volume']
			data = data.sort_values(by = ['Symbol',"Date"],ascending = ascending)	

	return data 



In [32]:
#Converting df_MMM_d amd df_MMM_m to type_1 data
df_MMM_m_type_1 = get_data_presentation_type_1(df_MMM_m,ascending = True)
df_MMM_d_type_1 = get_data_presentation_type_1(df_MMM_d,ascending = True)

In [33]:
# Showing df_MMM_m_type_1
df_MMM_m_type_1

,Unnamed: 0,Unnamed: 0.1,Date,TotalQuantity,TotalTradeCount,Time,Symbol,Open,Close,High,Low,Volume
24774,11443356,86394,20210412,200,1,04:42:00,MMM,198.00,198.00,198.00,198.00,39600.00
24775,11443357,86395,20210412,200,2,06:14:00,MMM,198.00,198.00,198.00,198.00,39600.00
24776,11443358,86396,20210412,100,1,08:44:00,MMM,197.87,197.87,197.87,197.87,19787.00
24777,11443359,86397,20210412,126,1,08:56:00,MMM,198.70,198.70,198.70,198.70,25036.20
24778,11443360,86398,20210412,12547,56,09:31:00,MMM,198.25,197.63,198.34,197.54,2485331.41
...,...,...,...,...,...,...,...,...,...,...,...,...
389,48114,48114,20210709,14470,108,15:59:00,MMM,201.11,201.04,201.11,200.99,2909039.61
390,48115,48115,20210709,24579,175,16:00:00,MMM,201.02,200.98,201.08,200.95,4940895.48
391,48116,48116,20210709,345187,26,16:04:00,MMM,201.00,201.00,201.00,201.00,69382587.00
392,48117,48117,20210709,133,1,16:59:00,MMM,201.00,201.00,201.00,201.00,26733.00


In [34]:
# Showing df_MMM_d_type_1
df_MMM_d_type_1

,Unnamed: 0,Unnamed: 0.1,Date,Symbol,Open,Close,High,Low,Volume
4439,2661673,289,19980102,MMM,41.09375,41.75000,41.8125,40.87500,2573000
5523,3309465,289,19980105,MMM,41.50000,41.68750,42.6250,41.34375,2505600
5907,3539823,289,19980106,MMM,41.62500,41.81250,42.8750,41.50000,3620200
5603,3357216,289,19980107,MMM,41.90625,42.15625,42.3125,41.37500,2494800
431,257054,289,19980108,MMM,42.00000,41.65625,42.0000,41.12500,1951400
...,...,...,...,...,...,...,...,...,...
76,45370,364,20210817,MMM,202.00000,199.55000,202.0000,197.60000,2287954
5245,3144218,364,20210818,MMM,198.57000,196.51000,198.8100,196.31000,2116228
5502,3297044,364,20210819,MMM,195.00000,195.36000,196.2300,194.30700,2312780
5118,3068265,364,20210820,MMM,194.34000,194.16000,195.1300,193.36500,2397425


In [ ]:
# A good practice that will later be implimented is to have all columns in lower case
# The current type_1 data does not by this will help when writing new code and consistency.
# Another good practice is to have all data rounded off to 2 decimal places.

In [35]:
# Let us remove columns we do not want
# For Minute Data
df_MMM_m_type_1.drop(['Unnamed: 0'], axis=1, inplace=True)
df_MMM_m_type_1.drop(['Unnamed: 0.1'], axis=1, inplace=True)
df_MMM_m_type_1.drop(['TotalQuantity'], axis=1, inplace=True)
df_MMM_m_type_1.drop(['TotalTradeCount'], axis=1, inplace=True)

# For Daily Data
df_MMM_d_type_1.drop(['Unnamed: 0'], axis=1, inplace=True)
df_MMM_d_type_1.drop(['Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
# Let us now take stock of the column types, 
# this is important because indicator code is written in a way that respects these types.

In [36]:
df_MMM_d_type_1.dtypes

Date        int64
Symbol     object
Open      float64
Close     float64
High      float64
Low       float64
Volume      int64
dtype: object

In [37]:
df_MMM_m_type_1.dtypes

Date        int64
Time       object
Symbol     object
Open      float64
Close     float64
High      float64
Low       float64
Volume    float64
dtype: object

In [38]:
# Now that our data is clean we will create different timeframes.
# All data is derived from minute data and the standard time frames are as follows:
# 1 min, 5 min, 15 min, 30 min, 409 mins, 1 hour, 4 hours, 
# 1 day or daily, 2 day, 1 week, monthly, quaterly, yearly, 5 years and 10 years.

In [67]:
from datetime import date, timedelta, datetime
from dateutil.parser import parse
 	 
def get_start_end_only_data_custom(data,start_date_, end_date_=None):
	"""Gets all data between two given dates 

	Args: 
	data :What data to perform the calculations on 
	start_date : The start date for the data 
	end_date : the end date for the data"""
	if start_date_:
		start_date = datetime(start_date_.year,start_date_.month,start_date_.day)
	if end_date_:
		end_date = 	datetime(end_date_.year,end_date_.month,end_date_.day)
	if "Time" not in data.columns : 
		Date =list(data.Date)
		date = []
		for _ in Date: 
			_ = str(_)
			date.append(f'{_[:4]}-{_[4:6]}-{_[6:]}')
		try :
			data.Date  = pd.to_datetime(date) 
		except: 
			pass
		if end_date :
			mask = (data['Date'] >= start_date) & (data['Date'] <= end_date)
		else: 
			mask = (data['Date'] >= start_date)
		return data.loc[mask]
	else: 
		try : 
			data.Time = data.Time.map(lambda x: datetime.strptime(x,"%H:%M:%S").time())
		except: pass
		Date =list(data.Date)
		start_time  = start_date_.time()
		date = []
		for _ in Date: 
			_ = str(_)
			date.append(f'{_[:4]}-{_[4:6]}-{_[6:]}')
		try: 
			data.Date  = pd.to_datetime(date) 
		except:pass
		if end_date :
			end_time  = end_date_.time()
			mask = (data['Date'] >= start_date) & (data['Date'] <= end_date ) &( data['Time'] >= start_time) & (data['Time'] <  end_time)
		else: 
			mask = (data['Date'] >= start_date) & ( data['Time'] >= start_time)
		return data.loc[mask]
    
def get_time_with_given_interval(date,interval,last_hour=None,last_minute=None):
	"""Gets time from 9:00:00 to 16:00:00 with given interval"""

	date  = parse(str(date))
	start_time =  datetime(date.year,date.month,date.day,9,30)
	if  last_hour==None : 
		end_time  =datetime(date.year,date.month,date.day,16)
	else :
		end_time  =datetime(date.year,date.month,date.day,16,last_hour,last_minute)
	list_of_times = []
	while start_time!=end_time :
		list_of_times.append([start_time,start_time+ timedelta(minutes = interval)])
		start_time  = start_time+ timedelta(minutes = interval)
		if start_time <=end_time :
			continue
		else: 
			break
	return list_of_times

def getconvert_minuteto_traditional  (data, convert_type,  trading_hours_criteria,last_hour= None,last_minute= None):
	"""Converts minute data to 5 minute , 15 minute , 30 minute , 1 hour , 2 hour and 5 hour data type 

	Arguments : 
	data : data to work with
	convert_type : what type to convert to 
	trading_hours_criteria : wether to work within trading hours or with all data"""

	dataframe_final ={
		"Date":[],
		"Time":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		"TotalQuantity":[],
		"TotalTradeCount":[]
		}
	if trading_hours_criteria == False :

		symbols = data.Symbol.unique()
		for symbol in symbols : 
			symbol_df = data[data['Symbol'] == symbol]
			dates = symbol_df.Date.unique()
			for date in dates : 
				date_symbol_df = symbol_df[symbol_df['Date'] == date]
				for step in range( convert_type,len(date_symbol_df),  convert_type):
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-convert_type:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalTradeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
				if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalrTfivadeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
	elif trading_hours_criteria== True :
		data.Time =data.Time.astype(str)
		if last_hour ==None : 
			data = data[data['Time'] >= "09:30:00" ]
			data = data[data['Time'] <= "16:00:00"]
		else:
			if len(str(last_minute))==1 :
				last_minute_ = "0"+str(last_minute)
			data = data[data['Time'] >= "09:30:00" ]
			data = data[data['Time'] <= f"{last_hour}:{last_minute_}:00"]			
		symbols = data.Symbol.unique()
		for symbol in symbols : 
			symbol_df = data[data['Symbol'] == symbol]
			dates = symbol_df.Date.unique()
			for date in dates : 

				date_symbol_df = symbol_df[symbol_df['Date'] == date]
				list_of_times_= get_time_with_given_interval(date,convert_type,last_hour,last_minute)
				for time in list_of_times_:
					current_dataframe = get_start_end_only_data_custom(date_symbol_df,time[0],time[1])
					try:
						dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
						dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
						dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
						dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
						dataframe_final["High"].append(max(current_dataframe['High']))
						dataframe_final["Low"].append(min(current_dataframe['Low']))
						dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
						if "TotalQuantity" in current_dataframe.columns : 

							dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
						else : 
							dataframe_final['TotalQuantity'].append(None)
						if "TotalTradeCount" in current_dataframe.columns : 

							dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
						else : 
							dataframe_final["TotalTradeCount"].append(None)
						dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
					except:
						pass	

    #convert to dataframe
	df_minute = pd.DataFrame(dataframe_final)
	return df_minute
    
def getconvert_minuteto_mathametical (data, convert_type,  trading_hours_criteria):
	"""Converts minute data to 5 minute , 15 minute , 30 minute , 1 hour , 2 hour and 5 hour data type 
	
	Arguments : 
	data : data to work with
	convert_type : what type to convert to 
	trading_hours_criteria : wether to work within trading hours or with all data"""

	dataframe_final ={
		"Date":[],
		"Time":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		"TotalQuantity":[],
		"TotalTradeCount":[]
		}
	if trading_hours_criteria == False :

		symbols = data.Symbol.unique()
		for symbol in symbols : 
			symbol_df = data[data['Symbol'] == symbol]
			dates = symbol_df.Date.unique()
			for date in dates : 
				date_symbol_df = symbol_df[symbol_df['Date'] == date]
				for step in range( convert_type,len(date_symbol_df),  convert_type):
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-convert_type:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalTradeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
				if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalTradeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
	elif trading_hours_criteria== True :
		data.Time =data.Time.astype(str)
		data = data[data['Time'] >= "09:30:00" ]
		data = data[data['Time'] <= "16:00:00"]
		symbols = data.Symbol.unique()
		for symbol in symbols : 
			symbol_df = data[data['Symbol'] == symbol]
			dates = symbol_df.Date.unique()
			for date in dates : 
				date_symbol_df = symbol_df[symbol_df['Date'] == date]
				for step in range( convert_type,len(date_symbol_df),  convert_type):
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-convert_type:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalTradeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
				if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Time'].append(current_dataframe['Time'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					if "TotalQuantity" in current_dataframe.columns : 

						dataframe_final["TotalQuantity"].append(sum(current_dataframe['TotalQuantity']))
					else : 
						dataframe_final['TotalQuantity'].append(None)
					if "TotalTradeCount" in current_dataframe.columns : 

						dataframe_final["TotalTradeCount"].append(sum(current_dataframe['TotalTradeCount']))
					else : 
						dataframe_final["TotalTradeCount"].append(None)
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
    #convert to dataframe
	df_minute = pd.DataFrame(dataframe_final)
	return df_minute

    
def getconvert_dailyto_mathamtical(data,amount):
	"""Converts daily data to different types of data
	Arguments : 
 	data : data to work with
	amount : what frequencey to work with """
	
	year = []
	month = []
	week = []
	day = []
	dataframe_final ={
		"Date":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		}
	for date in data.Date:
		date = str(date)
		date_time_obj = datetime.strptime(date, '%Y%m%d')
		week.append(date_time_obj.date().isocalendar()[1])
		month.append(date_time_obj.month)
		year.append(date_time_obj.year)
		day.append(date_time_obj.isoweekday())
	data['Month'] = month
	data["Year"] = year
	data['Week'] = week
	data["Day"]  = day
	symbols = data.Symbol.unique()
	for symbol in symbols : 
			date_symbol_df = data[data['Symbol'] == symbol]
			for step in range( amount,len(date_symbol_df),  amount): 
				remaining = len(date_symbol_df)- step 
				current_dataframe = date_symbol_df.iloc[step-amount:step]
				dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
				dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
				dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
				dataframe_final["High"].append(max(current_dataframe['High']))
				dataframe_final["Low"].append(min(current_dataframe['Low']))
				dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
				dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
			if remaining != 0 : 
				current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
				dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
				dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
				dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
				dataframe_final["High"].append(max(current_dataframe['High']))
				dataframe_final["Low"].append(min(current_dataframe['Low']))
				dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
				dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
    #convert to dataframe
	df_daily = pd.DataFrame(dataframe_final)
	return df_daily
    
def getconvert_dailyto_traditional(data,amount,convert_type):
	"""Converts daily data to ""Daily" "Weekly" "Monthly" "Yearly"
	Arguments : 
 	data : data to work with
	amount : what frequencey to work with
	convert_type : what type to convert to  """
	year = []
	month = []
	week = []
	dataframe_final ={
		"Date":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		}
	for date in data.Date:
		date = str(date)
		date_time_obj = datetime.strptime(date, '%Y%m%d')
		week.append(date_time_obj.date().isocalendar()[1])
		month.append(date_time_obj.month)
		year.append(date_time_obj.year)
	data['Month'] = month
	data["Year"] = year
	data['Week'] = week

	symbols = data.Symbol.unique()
	if convert_type == "Daily": 
		dataframe_final = pd.DataFrame(getconvert_dailyto_mathamtical(data,amount))
	elif convert_type == "Weekly":
		dataframe_weekly ={
		"Date":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		}
		for current_symbol in symbols :
			symbol_data = data[data["Symbol"]== current_symbol]
			years_we_have = symbol_data.Year.unique()
			for year in years_we_have : 
				data_for_current_year = symbol_data[symbol_data["Year"] == year]
				weeks_for_current_year = data_for_current_year.Week.unique()
				for week in weeks_for_current_year : 
					data_for_current_week_for_current_year = data_for_current_year[data_for_current_year['Week'] == week]
					data_for_current_week_for_current_year = data_for_current_week_for_current_year.sort_values(["Date"],ascending = True)
					dataframe_weekly['Date'].append(data_for_current_week_for_current_year['Date'].iloc[0])
					dataframe_weekly['Open'].append(data_for_current_week_for_current_year['Open'].iloc[0])
					dataframe_weekly['Close'].append(data_for_current_week_for_current_year['Close'].iloc[-1])
					dataframe_weekly["High"].append(max(data_for_current_week_for_current_year['High']))
					dataframe_weekly["Low"].append(min(data_for_current_week_for_current_year['Low']))
					dataframe_weekly["Volume"].append(sum(data_for_current_week_for_current_year['Volume']))
					dataframe_weekly['Symbol'].append(data_for_current_week_for_current_year['Symbol'].iloc[0])
		dataframe_weekly = pd.DataFrame(dataframe_weekly)
		for symbol in symbols : 
			date_symbol_df = dataframe_weekly[dataframe_weekly['Symbol'] == symbol]
			for step in range( amount,len(date_symbol_df),  amount): 
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-amount:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
			if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
	elif convert_type == "Monthly":
		dataframe_monthly ={
		"Date":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		}
		for current_symbol in symbols :
			symbol_data = data[data["Symbol"]== current_symbol]
			years_we_have = symbol_data.Year.unique()
			for year in years_we_have : 
				data_for_current_year = symbol_data[symbol_data["Year"] == year]
				months_for_current_year = data_for_current_year.Month.unique()
				for month in months_for_current_year : 
					data_for_current_month_for_current_year = data_for_current_year[data_for_current_year['Month'] == month]
					data_for_current_month_for_current_year = data_for_current_month_for_current_year.sort_values(["Date"],ascending = True)
					dataframe_monthly['Date'].append(data_for_current_month_for_current_year['Date'].iloc[0])
					dataframe_monthly['Open'].append(data_for_current_month_for_current_year['Open'].iloc[0])
					dataframe_monthly['Close'].append(data_for_current_month_for_current_year['Close'].iloc[-1])
					dataframe_monthly["High"].append(max(data_for_current_month_for_current_year['High']))
					dataframe_monthly["Low"].append(min(data_for_current_month_for_current_year['Low']))
					dataframe_monthly["Volume"].append(sum(data_for_current_month_for_current_year['Volume']))
					dataframe_monthly['Symbol'].append(data_for_current_month_for_current_year['Symbol'].iloc[0])
		dataframe_monthly = pd.DataFrame(dataframe_monthly)
		for symbol in symbols : 
			date_symbol_df = dataframe_monthly[dataframe_monthly['Symbol'] == symbol]
			for step in range( amount,len(date_symbol_df),  amount): 
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-amount:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
			if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
	elif convert_type == "Yearly":
		dataframe_Yearly ={
		"Date":[],
		"Open":[],
		"Close":[],
		"High":[],
		"Low":[],
		"Volume":[],
		"Symbol":[],
		}
		for current_symbol in symbols :
			symbol_data = data[data["Symbol"]== current_symbol]
			years_we_have = symbol_data.Year.unique()
			for year in years_we_have : 
				data_for_current_year = symbol_data[symbol_data["Year"] == year]
				data_for_current_year = data_for_current_year.sort_values(["Date"],ascending = True)
				dataframe_Yearly['Date'].append(data_for_current_year['Date'].iloc[0])
				dataframe_Yearly['Open'].append(data_for_current_year['Open'].iloc[0])
				dataframe_Yearly['Close'].append(data_for_current_year['Close'].iloc[-1])
				dataframe_Yearly["High"].append(max(data_for_current_year['High']))
				dataframe_Yearly["Low"].append(min(data_for_current_year['Low']))
				dataframe_Yearly["Volume"].append(sum(data_for_current_year['Volume']))
				dataframe_Yearly['Symbol'].append(data_for_current_year['Symbol'].iloc[0])
		dataframe_Yearly = pd.DataFrame(dataframe_Yearly)
		for symbol in symbols : 
			date_symbol_df = dataframe_Yearly[dataframe_Yearly['Symbol'] == symbol]
			for step in range( amount,len(date_symbol_df),  amount): 
					remaining = len(date_symbol_df)- step 
					current_dataframe = date_symbol_df.iloc[step-amount:step]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
			if remaining != 0 : 
					current_dataframe = date_symbol_df[len(date_symbol_df)-remaining : ]
					dataframe_final['Date'].append(current_dataframe['Date'].iloc[0])
					dataframe_final['Open'].append(current_dataframe['Open'].iloc[0])
					dataframe_final['Close'].append(current_dataframe['Close'].iloc[-1])
					dataframe_final["High"].append(max(current_dataframe['High']))
					dataframe_final["Low"].append(min(current_dataframe['Low']))
					dataframe_final["Volume"].append(sum(current_dataframe['Volume']))
					dataframe_final['Symbol'].append(current_dataframe['Symbol'].iloc[0])
    #convert to dataframe
	df_daily = pd.DataFrame(dataframe_final)
	return df_daily

In [70]:
# The timeframe is another stage of cleaning the data.
# The Stock market is open Monday to Friday, excluding holidays from 9:30 to 4:00pm EST
# There are some exceptions in which it is open half days to 1:00pm EST

# So depending on how we want to analyse the data we can trim off data before 9:30am and after 4:00pm


In [56]:
df_MMM_m_type_1

,Date,Time,Symbol,Open,Close,High,Low,Volume
24774,20210412,04:42:00,MMM,198.00,198.00,198.00,198.00,39600.00
24775,20210412,06:14:00,MMM,198.00,198.00,198.00,198.00,39600.00
24776,20210412,08:44:00,MMM,197.87,197.87,197.87,197.87,19787.00
24777,20210412,08:56:00,MMM,198.70,198.70,198.70,198.70,25036.20
24778,20210412,09:31:00,MMM,198.25,197.63,198.34,197.54,2485331.41
...,...,...,...,...,...,...,...,...
389,20210709,15:59:00,MMM,201.11,201.04,201.11,200.99,2909039.61
390,20210709,16:00:00,MMM,201.02,200.98,201.08,200.95,4940895.48
391,20210709,16:04:00,MMM,201.00,201.00,201.00,201.00,69382587.00
392,20210709,16:59:00,MMM,201.00,201.00,201.00,201.00,26733.00


In [81]:
# Lets get a 1 min from the minute data
# This code take a long time for just MMM I can not imagine how long it will take for 500 - 5000 symbols
df_MMM_1m_mathametical = getconvert_minuteto_mathametical (df_MMM_m_type_1, 1,  True)
df_MMM_1m_mathametical

,Date,Time,Open,Close,High,Low,Volume,Symbol,TotalQuantity,TotalTradeCount
0,20210412,09:31:00,198.25,197.63,198.34,197.54,2485331.41,MMM,None,None
1,20210412,09:32:00,197.70,198.26,198.26,197.70,877144.31,MMM,None,None
2,20210412,09:33:00,198.26,198.14,198.26,198.04,477146.81,MMM,None,None
3,20210412,09:34:00,198.12,197.92,198.15,197.91,297396.62,MMM,None,None
4,20210412,09:35:00,198.02,198.44,198.44,198.01,800694.64,MMM,None,None
...,...,...,...,...,...,...,...,...,...,...
24495,20210709,15:56:00,201.04,201.04,201.07,200.96,2176869.20,MMM,None,None
24496,20210709,15:57:00,201.07,201.05,201.10,201.03,2058977.34,MMM,None,None
24497,20210709,15:58:00,201.07,201.11,201.11,201.04,3037745.19,MMM,None,None
24498,20210709,15:59:00,201.11,201.04,201.11,200.99,2909039.61,MMM,None,None


In [131]:
# Lets get a 1 min from the minute data
# Traditional groups time frames from 0
# Bug Alert - The trading_hours_criteria = False is not working
# This code take a long time for just MMM I can not imagine how long it will take for 500 - 5000 symbols
# Note - Code adds unncessary columns: TotalQuantity	TotalTradeCount
df_MMM_1m_traditional = getconvert_minuteto_traditional  (df_MMM_m_type_1, 1,  trading_hours_criteria = True,last_hour= 4,last_minute= 0)
df_MMM_1m_traditional

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Date,Time,Open,Close,High,Low,Volume,Symbol,TotalQuantity,TotalTradeCount
0,2021-04-12,09:31:00,198.25,197.63,198.34,197.54,2485331.41,MMM,None,None
1,2021-04-12,09:32:00,197.70,198.26,198.26,197.70,877144.31,MMM,None,None
2,2021-04-12,09:33:00,198.26,198.14,198.26,198.04,477146.81,MMM,None,None
3,2021-04-12,09:34:00,198.12,197.92,198.15,197.91,297396.62,MMM,None,None
4,2021-04-12,09:35:00,198.02,198.44,198.44,198.01,800694.64,MMM,None,None
...,...,...,...,...,...,...,...,...,...,...
24524,2021-07-09,15:56:00,201.04,201.04,201.07,200.96,2176869.20,MMM,None,None
24525,2021-07-09,15:57:00,201.07,201.05,201.10,201.03,2058977.34,MMM,None,None
24526,2021-07-09,15:58:00,201.07,201.11,201.11,201.04,3037745.19,MMM,None,None
24527,2021-07-09,15:59:00,201.11,201.04,201.11,200.99,2909039.61,MMM,None,None


In [130]:
# Lets get a 5 min from the minute data
df_MMM_5m_traditional = getconvert_minuteto_traditional  (df_MMM_m_type_1, 5,  trading_hours_criteria = True,last_hour= 4,last_minute= 4)
df_MMM_5m_traditional

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Date,Time,Open,Close,High,Low,Volume,Symbol,TotalQuantity,TotalTradeCount
0,2021-04-12,09:31:00,198.25,197.92,198.34,197.54,4137019.15,MMM,None,None
1,2021-04-12,09:35:00,198.02,198.18,198.44,198.01,2505193.36,MMM,None,None
2,2021-04-12,09:40:00,198.06,198.76,198.84,197.90,4286892.02,MMM,None,None
3,2021-04-12,09:45:00,198.79,198.70,199.25,198.61,7802217.73,MMM,None,None
4,2021-04-12,09:50:00,198.84,199.16,199.21,198.72,3239573.69,MMM,None,None
...,...,...,...,...,...,...,...,...,...,...
4972,2021-07-09,15:40:00,201.00,200.95,201.06,200.93,5200669.96,MMM,None,None
4973,2021-07-09,15:45:00,200.95,201.03,201.04,200.88,3869397.91,MMM,None,None
4974,2021-07-09,15:50:00,201.05,201.12,201.14,201.02,5862105.59,MMM,None,None
4975,2021-07-09,15:55:00,201.13,201.04,201.13,200.96,11167800.62,MMM,None,None


In [129]:
# Lets get a 15 min from the minute data
# Bug Alert - Code will not accept last_minute > 10 only 9 and below
df_MMM_15m_traditional = getconvert_minuteto_traditional  (df_MMM_m_type_1, 15,  trading_hours_criteria = True,last_hour= 4,last_minute= 9)
df_MMM_15m_traditional

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Date,Time,Open,Close,High,Low,Volume,Symbol,TotalQuantity,TotalTradeCount
0,2021-04-12,09:31:00,198.25,198.76,198.84,197.54,10929104.53,MMM,None,None
1,2021-04-12,09:45:00,198.79,199.10,199.27,198.61,15022796.14,MMM,None,None
2,2021-04-12,10:00:00,199.13,198.41,199.29,198.35,6986501.49,MMM,None,None
3,2021-04-12,10:15:00,198.46,198.35,198.68,198.27,5175655.26,MMM,None,None
4,2021-04-12,10:30:00,198.33,198.57,198.67,198.20,6350619.87,MMM,None,None
...,...,...,...,...,...,...,...,...,...,...
1696,2021-07-09,15:00:00,201.06,201.11,201.23,201.00,4411411.36,MMM,None,None
1697,2021-07-09,15:15:00,201.09,201.08,201.18,201.01,5417800.80,MMM,None,None
1698,2021-07-09,15:30:00,201.10,200.95,201.10,200.93,10370560.40,MMM,None,None
1699,2021-07-09,15:45:00,200.95,201.04,201.14,200.88,20899304.12,MMM,None,None


In [128]:
# Lets get a 1 hour from the minute data
df_MMM_1h_traditional = getconvert_minuteto_traditional  (df_MMM_m_type_1, 60,  trading_hours_criteria = True,last_hour= 4,last_minute= 9)
df_MMM_1h_traditional

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Date,Time,Open,Close,High,Low,Volume,Symbol,TotalQuantity,TotalTradeCount
0,2021-04-12,09:31:00,198.25,198.35,199.29,197.54,3.811406e+07,MMM,None,None
1,2021-04-12,10:30:00,198.33,198.22,198.67,197.82,2.944195e+07,MMM,None,None
2,2021-04-12,11:30:00,198.25,198.15,198.73,198.06,2.348289e+07,MMM,None,None
3,2021-04-12,12:30:00,198.15,197.59,198.39,197.39,2.211164e+07,MMM,None,None
4,2021-04-12,13:30:00,197.58,197.28,197.61,197.06,1.912919e+07,MMM,None,None
...,...,...,...,...,...,...,...,...,...,...
436,2021-07-09,11:30:00,201.01,200.53,201.52,200.48,1.818551e+07,MMM,None,None
437,2021-07-09,12:30:00,200.56,200.86,200.92,200.45,1.463036e+07,MMM,None,None
438,2021-07-09,13:30:00,200.87,200.99,201.06,200.62,1.778228e+07,MMM,None,None
439,2021-07-09,14:30:00,201.01,201.08,201.23,200.87,2.486125e+07,MMM,None,None


In [97]:
# Lets get a 2 Day min from Daily data
# Note - There is no reason to do a 1 Day because that is daily data
df_MMM_2d_mathametical = getconvert_dailyto_mathamtical(df_MMM_d_type_1,2)
df_MMM_2d_mathametical

,Date,Open,Close,High,Low,Volume,Symbol
0,19980102,41.09375,41.68750,42.6250,40.8750,5078600,MMM
1,19980106,41.62500,42.15625,42.8750,41.3750,6115000,MMM
2,19980108,42.00000,41.31250,42.0000,41.0000,9934400,MMM
3,19980112,40.50000,42.06250,42.0625,40.5000,4177400,MMM
4,19980114,41.87500,41.25000,42.1250,41.1875,3124800,MMM
...,...,...,...,...,...,...,...
2970,20210810,198.24000,201.57000,201.7700,197.7000,3815880,MMM
2971,20210812,201.48000,200.58000,202.3728,200.2900,3142884,MMM
2972,20210816,201.09000,199.55000,202.7700,197.6000,3895054,MMM
2973,20210818,198.57000,195.36000,198.8100,194.3070,4429008,MMM


In [98]:
# Lets get a 2 Day from Daily data in a Traditional way
# Converts daily data types are Daily" "Weekly" "Monthly" "Yearly"
# Traditional is suppose to start from the beginning of the week (1st day of the week)
df_MMM_2d_traditional = getconvert_dailyto_traditional(df_MMM_d_type_1,2,"Daily")
df_MMM_2d_traditional

,Date,Open,Close,High,Low,Volume,Symbol
0,19980102,41.09375,41.68750,42.6250,40.8750,5078600,MMM
1,19980106,41.62500,42.15625,42.8750,41.3750,6115000,MMM
2,19980108,42.00000,41.31250,42.0000,41.0000,9934400,MMM
3,19980112,40.50000,42.06250,42.0625,40.5000,4177400,MMM
4,19980114,41.87500,41.25000,42.1250,41.1875,3124800,MMM
...,...,...,...,...,...,...,...
2970,20210810,198.24000,201.57000,201.7700,197.7000,3815880,MMM
2971,20210812,201.48000,200.58000,202.3728,200.2900,3142884,MMM
2972,20210816,201.09000,199.55000,202.7700,197.6000,3895054,MMM
2973,20210818,198.57000,195.36000,198.8100,194.3070,4429008,MMM


In [99]:
# Lets get a weekly from Daily data in a Traditional way
# Converts daily data types are Daily" "Weekly" "Monthly" "Yearly"
# Traditional is suppose to start from the beginning of the week (1st day of the week)
df_MMM_1w_traditional = getconvert_dailyto_traditional(df_MMM_d_type_1,1,"Weekly")
df_MMM_1w_traditional

,Date,Open,Close,High,Low,Volume,Symbol
0,19980102,41.09375,41.75000,41.81250,40.87500,2573000,MMM
1,19980105,41.50000,41.31250,42.87500,41.00000,18555000,MMM
2,19980112,40.50000,41.62500,42.12500,40.50000,9829200,MMM
3,19980120,41.62500,40.78125,41.75000,40.03125,8235000,MMM
4,19980126,41.00000,41.75000,42.96875,40.28125,13163200,MMM
...,...,...,...,...,...,...,...
1230,20210726,200.45000,197.94000,202.74000,194.91000,11579084,MMM
1231,20210802,199.07000,198.13000,202.21000,197.20000,8289593,MMM
1232,20210809,197.65000,200.58000,202.37280,196.78000,8152122,MMM
1233,20210816,201.09000,194.16000,202.77000,193.36500,10721487,MMM


In [100]:
# Lets get a monthly from Daily data in a Traditional way
# Converts daily data types are Daily" "Weekly" "Monthly" "Yearly"
# Traditional is suppose to start from the beginning of the week (1st day of the week)
df_MMM_1mon_traditional = getconvert_dailyto_traditional(df_MMM_d_type_1,1,"Monthly")
df_MMM_1mon_traditional

,Date,Open,Close,High,Low,Volume,Symbol
0,19980102,41.09375,41.75000,42.96875,40.03125,52355400,MMM
1,19980202,42.18750,42.65625,44.68750,42.15625,34888400,MMM
2,19980302,42.65625,45.50000,48.06250,42.65625,45769800,MMM
3,19980401,46.31250,47.18750,48.93750,44.53125,36851600,MMM
4,19980501,47.18750,46.31250,48.75000,46.21875,26461800,MMM
...,...,...,...,...,...,...,...
279,20210401,193.25000,197.14000,203.15500,188.63000,46438173,MMM
280,20210503,197.99000,203.04000,208.95000,196.66000,37466515,MMM
281,20210601,204.98000,198.63000,206.80630,191.53000,51315729,MMM
282,20210701,199.45000,197.94000,203.21000,194.76000,46028249,MMM


In [105]:
# Now that we have our data in the time frames we want let us now add columns
# There are x types of columns where x = = 8 for now
# 1. Indicators e.g. moving averages, slope, etc the are 100s of columns that are to contain indicator returns
# 2. Profile Data e.g. CIK, Company name etc
# 3. Accounting Data e.g. Revenues, cash flow, earnings
# 4. Accounting Ratios e.g. Apply formulas to Accounting Data
# 5. Propietary Data e.g. Data from Short Squeeze
# 6. Missing Data e.g. Data obtained from Bloomberg
# 7. Experimental Data e.g creating new symbols derived for current groups of symbols
# 8. Predictive Data e.g. Using Blast

In [ ]:
# Let us add indicators

In [ ]:
# Custom indicators group 1
# These are simple indicators 

In [109]:
def SimpleMovingAvg(data, price, length):
	symbols = data["Symbol"].unique()
	sma = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].rolling(length).mean():
			sma.append(__)
	return sma

def ExpMovingAvg(data, price, length):
	symbols = data["Symbol"].unique()
	ema = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].ewm(span=length, adjust=False).mean():
			ema.append(__)
	return ema

def movingAverageConvergenceDivergence_Value(data, price,fastLength, slowLength, MACDLength):
	symbols = data["Symbol"].unique()
	Value = []
	Average = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].ewm(span=fastLength, adjust=False).mean() - data[data['Symbol'] == _][price].ewm(span=slowLength, adjust=False).mean():
			Value.append(__)
		for __ in data[data['Symbol'] == _][price].ewm(span=MACDLength, adjust=False).mean():
			Average.append(__)
	return Value

def movingAverageConvergenceDivergence_Average(data, price,fastLength, slowLength, MACDLength):
	symbols = data["Symbol"].unique()
	Value = []
	Average = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].ewm(span=fastLength, adjust=False).mean() - data[data['Symbol'] == _][price].ewm(span=slowLength, adjust=False).mean():
			Value.append(__)
		for __ in data[data['Symbol'] == _][price].ewm(span=MACDLength, adjust=False).mean():
			Average.append(__)
	return Average

def movingAverageConvergenceDivergence_Difference(data, price,fastLength, slowLength, MACDLength):
	symbols = data["Symbol"].unique()
	Value = []
	Average = []
	Difference = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].ewm(span=fastLength, adjust=False).mean() - data[data['Symbol'] == _][price].ewm(span=slowLength, adjust=False).mean():
			Value.append(__)
		for __ in data[data['Symbol'] == _][price].ewm(span=MACDLength, adjust=False).mean():
			Average.append(__)
	for i in range(len(Value)):
		Difference.append(Value[i] - Average[i])
	return Difference

In [107]:
# We will use the 15min time frame  (df_MMM_15m_traditional) and the weekly (df_MMM_1w_traditional)

In [113]:
# Cleaning
df_MMM_15m_traditional.drop(['TotalQuantity'], axis=1, inplace=True)
df_MMM_15m_traditional.drop(['TotalTradeCount'], axis=1, inplace=True)

In [114]:
# Applying Indicators on the 15min time frame
df_MMM_15m = df_MMM_15m_traditional
df_MMM_15m["2_p_sma"] = SimpleMovingAvg(df_MMM_15m_traditional, "Close", 2)
df_MMM_15m["macd_val"]= movingAverageConvergenceDivergence_Value(df_MMM_15m_traditional, "Close",12, 26, 9)
df_MMM_15m["macd_avg"] = movingAverageConvergenceDivergence_Average(df_MMM_15m_traditional, "Close",12, 26, 9)
df_MMM_15m["macd_diff"] = movingAverageConvergenceDivergence_Difference(df_MMM_15m_traditional, "Close",12, 26, 9)
df_MMM_15m

,Date,Time,Open,Close,High,Low,Volume,Symbol,2_p_sma,macd_val,macd_avg,macd_diff
0,2021-04-12,09:31:00,198.25,198.35,199.29,197.54,3.811406e+07,MMM,NaN,0.000000,198.350000,-198.350000
1,2021-04-12,10:30:00,198.33,198.22,198.67,197.82,2.944195e+07,MMM,198.285,-0.010370,198.324000,-198.334370
2,2021-04-12,11:30:00,198.25,198.15,198.73,198.06,2.348289e+07,MMM,198.185,-0.023961,198.289200,-198.313161
3,2021-04-12,12:30:00,198.15,197.59,198.39,197.39,2.211164e+07,MMM,197.870,-0.079009,198.149360,-198.228369
4,2021-04-12,13:30:00,197.58,197.28,197.61,197.06,1.912919e+07,MMM,197.435,-0.145966,197.975488,-198.121454
...,...,...,...,...,...,...,...,...,...,...,...,...
436,2021-07-09,11:30:00,201.01,200.53,201.52,200.48,1.818551e+07,MMM,200.800,0.592100,199.620256,-199.028156
437,2021-07-09,12:30:00,200.56,200.86,200.92,200.45,1.463036e+07,MMM,200.695,0.665545,199.868205,-199.202660
438,2021-07-09,13:30:00,200.87,200.99,201.06,200.62,1.778228e+07,MMM,200.925,0.725873,200.092564,-199.366691
439,2021-07-09,14:30:00,201.01,201.08,201.23,200.87,2.486125e+07,MMM,201.035,0.772046,200.290051,-199.518005


In [116]:
# Applying Indicators on the 1 week time frame
df_MMM_1w = df_MMM_1w_traditional
df_MMM_1w["2_p_sma"] = SimpleMovingAvg(df_MMM_1w_traditional, "Close", 2)
df_MMM_1w["macd_val"]= movingAverageConvergenceDivergence_Value(df_MMM_1w_traditional, "Close",12, 26, 9)
df_MMM_1w["macd_avg"] = movingAverageConvergenceDivergence_Average(df_MMM_1w_traditional, "Close",12, 26, 9)
df_MMM_1w["macd_diff"] = movingAverageConvergenceDivergence_Difference(df_MMM_1w_traditional, "Close",12, 26, 9)
df_MMM_1w

,Date,Open,Close,High,Low,Volume,Symbol,2_p_sma,macd_val,macd_avg,macd_diff
0,19980102,41.09375,41.75000,41.81250,40.87500,2573000,MMM,NaN,0.000000,41.750000,-41.750000
1,19980105,41.50000,41.31250,42.87500,41.00000,18555000,MMM,41.531250,-0.034900,41.662500,-41.697400
2,19980112,40.50000,41.62500,42.12500,40.50000,9829200,MMM,41.468750,-0.036917,41.655000,-41.691917
3,19980120,41.62500,40.78125,41.75000,40.03125,8235000,MMM,41.203125,-0.105385,41.480250,-41.585635
4,19980126,41.00000,41.75000,42.96875,40.28125,13163200,MMM,41.265625,-0.080547,41.534200,-41.614747
...,...,...,...,...,...,...,...,...,...,...,...
1230,20210726,200.45000,197.94000,202.74000,194.91000,11579084,MMM,199.215000,5.073759,199.079634,-194.005876
1231,20210802,199.07000,198.13000,202.21000,197.20000,8289593,MMM,198.035000,4.662252,198.889708,-194.227456
1232,20210809,197.65000,200.58000,202.37280,196.78000,8152122,MMM,199.355000,4.482157,199.227766,-194.745609
1233,20210816,201.09000,194.16000,202.77000,193.36500,10721487,MMM,197.370000,3.777841,198.214213,-194.436371


In [ ]:
# Custom indicators group 2
# These are difficult indicators - currently being coded

In [ ]:
# Next we will apply library indicators
# There are others but we will use talib

In [117]:
import talib as ta

In [118]:
"""
1. Use talib to calculate Doji
"""
def Doji(data, open, high, low, close):
	symbols = data["Symbol"].unique()
	Doji = []
	for _ in symbols: 
		for __ in ta.CDLDOJI(data[data['Symbol'] == _][open], data[data['Symbol'] == _][high], data[data['Symbol'] == _][low], data[data['Symbol'] == _][close]):
			Doji.append(__)
	return Doji

"""
1. Use talib to calculate Engulfing Pattern
"""
def EngulfingPattern(data, open, high, low, close):
	symbols = data["Symbol"].unique()
	EngulfingPattern = []
	for _ in symbols: 
		for __ in ta.CDLENGULFING(data[data['Symbol'] == _][open], data[data['Symbol'] == _][high], data[data['Symbol'] == _][low], data[data['Symbol'] == _][close]):
			EngulfingPattern.append(__)
	return EngulfingPattern

"""
1. Use talib to calculate RSI(data, price, timeperiod)
"""
def RSI(data, price, timeperiod):
	symbols = data["Symbol"].unique()
	RSI = []
	for _ in symbols: 
		for __ in ta.RSI(data[data['Symbol'] == _][price], timeperiod=timeperiod):
			RSI.append(__)
	return RSI


In [121]:
# Adding more indicators from libraries for 15min time frame
df_MMM_15m["doji"] = Doji(df_MMM_15m, "Open", "High", "Low", "Close")
df_MMM_15m["engulfing"] = EngulfingPattern(df_MMM_15m, "Open", "High", "Low", "Close")
df_MMM_15m["5_RSI"] = RSI(df_MMM_15m, "Close",5)
df_MMM_15m["14_RSI"] = RSI(df_MMM_15m, "Close",14)
df_MMM_15m

,Date,Time,Open,Close,High,Low,Volume,Symbol,2_p_sma,macd_val,macd_avg,macd_diff,doji,engulfing,5_RSI,14_RSI
0,2021-04-12,09:31:00,198.25,198.35,199.29,197.54,3.811406e+07,MMM,NaN,0.000000,198.350000,-198.350000,0,0,NaN,NaN
1,2021-04-12,10:30:00,198.33,198.22,198.67,197.82,2.944195e+07,MMM,198.285,-0.010370,198.324000,-198.334370,0,0,NaN,NaN
2,2021-04-12,11:30:00,198.25,198.15,198.73,198.06,2.348289e+07,MMM,198.185,-0.023961,198.289200,-198.313161,0,0,NaN,NaN
3,2021-04-12,12:30:00,198.15,197.59,198.39,197.39,2.211164e+07,MMM,197.870,-0.079009,198.149360,-198.228369,0,0,NaN,NaN
4,2021-04-12,13:30:00,197.58,197.28,197.61,197.06,1.912919e+07,MMM,197.435,-0.145966,197.975488,-198.121454,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,2021-07-09,11:30:00,201.01,200.53,201.52,200.48,1.818551e+07,MMM,200.800,0.592100,199.620256,-199.028156,0,0,64.218169,59.643224
437,2021-07-09,12:30:00,200.56,200.86,200.92,200.45,1.463036e+07,MMM,200.695,0.665545,199.868205,-199.202660,0,0,67.563941,61.048422
438,2021-07-09,13:30:00,200.87,200.99,201.06,200.62,1.778228e+07,MMM,200.925,0.725873,200.092564,-199.366691,0,0,68.991689,61.615434
439,2021-07-09,14:30:00,201.01,201.08,201.23,200.87,2.486125e+07,MMM,201.035,0.772046,200.290051,-199.518005,100,0,70.129512,62.027550


In [122]:
# Adding more indicators from libraries for 1 week time frame
df_MMM_1w["doji"] = Doji(df_MMM_1w, "Open", "High", "Low", "Close")
df_MMM_1w["engulfing"] = EngulfingPattern(df_MMM_1w, "Open", "High", "Low", "Close")
df_MMM_1w["5_RSI"] = RSI(df_MMM_1w, "Close",5)
df_MMM_1w["14_RSI"] = RSI(df_MMM_1w, "Close",14)
df_MMM_1w

,Date,Open,Close,High,Low,Volume,Symbol,2_p_sma,macd_val,macd_avg,macd_diff,doji,engulfing,5_RSI,14_RSI
0,19980102,41.09375,41.75000,41.81250,40.87500,2573000,MMM,NaN,0.000000,41.750000,-41.750000,0,0,NaN,NaN
1,19980105,41.50000,41.31250,42.87500,41.00000,18555000,MMM,41.531250,-0.034900,41.662500,-41.697400,0,0,NaN,NaN
2,19980112,40.50000,41.62500,42.12500,40.50000,9829200,MMM,41.468750,-0.036917,41.655000,-41.691917,0,100,NaN,NaN
3,19980120,41.62500,40.78125,41.75000,40.03125,8235000,MMM,41.203125,-0.105385,41.480250,-41.585635,0,0,NaN,NaN
4,19980126,41.00000,41.75000,42.96875,40.28125,13163200,MMM,41.265625,-0.080547,41.534200,-41.614747,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,20210726,200.45000,197.94000,202.74000,194.91000,11579084,MMM,199.215000,5.073759,199.079634,-194.005876,0,0,45.014476,55.474259
1231,20210802,199.07000,198.13000,202.21000,197.20000,8289593,MMM,198.035000,4.662252,198.889708,-194.227456,0,0,45.913595,55.664836
1232,20210809,197.65000,200.58000,202.37280,196.78000,8152122,MMM,199.355000,4.482157,199.227766,-194.745609,0,100,57.195461,58.152154
1233,20210816,201.09000,194.16000,202.77000,193.36500,10721487,MMM,197.370000,3.777841,198.214213,-194.436371,0,-100,33.979433,50.203864


In [124]:
# Let us see what columns we have for the df_MMM_15m
list(df_MMM_15m.columns.values)

['Date',
 'Time',
 'Open',
 'Close',
 'High',
 'Low',
 'Volume',
 'Symbol',
 '2_p_sma',
 'macd_val',
 'macd_avg',
 'macd_diff',
 'doji',
 'engulfing',
 '5_RSI',
 '14_RSI']

In [125]:
# Let us see what columns we have for the df_MMM_1w
list(df_MMM_1w.columns.values)

['Date',
 'Open',
 'Close',
 'High',
 'Low',
 'Volume',
 'Symbol',
 '2_p_sma',
 'macd_val',
 'macd_avg',
 'macd_diff',
 'doji',
 'engulfing',
 '5_RSI',
 '14_RSI']

In [ ]:
# Remember the List?
# There are x types of columns where x = = 8 for now
# 1. Indicators e.g. moving averages, slope, etc the are 100s of columns that are to contain indicator returns
# 2. Profile Data e.g. CIK, Company name etc
# 3. Accounting Data e.g. Revenues, cash flow, earnings
# 4. Accounting Ratios e.g. Apply formulas to Accounting Data
# 5. Propietary Data e.g. Data from Short Squeeze
# 6. Missing Data e.g. Data obtained from Bloomberg
# 7. Experimental Data e.g creating new symbols derived for current groups of symbols
# 8. Predictive Data e.g. Using Blast

In [127]:
# 3. Accounting Data e.g. Revenues, cash flow, earnings
# This data is retrived from financialmodelingprep


url = "https://financialmodelingprep.com/api/v3/ratios/MMM?period=quarter&limit=140&apikey=aa8b4631b2ad1b2704741d1a2d6a2611"

response = requests.get(url)
data = response.json()
df_MMM_accounting_all_data = pd.DataFrame(data)
df_MMM_accounting_all_data

,symbol,date,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,MMM,2022-03-31,Q1,1.579108,0.893138,0.354786,199.057085,98.653129,147.735698,59.005387,...,9.650473,16.398003,145.151659,84.276977,84.276977,-6.367891,9.650473,0.010000,45.791083,5.706900
1,MMM,2021-12-31,Q4,1.704815,1.055340,0.505147,202.165583,95.498084,145.347132,57.356322,...,11.857814,19.066372,70.475838,50.932414,50.932414,-3.363853,11.857814,0.008304,52.882469,6.787152
2,MMM,2021-09-30,Q3,1.704826,1.105241,0.506279,200.664281,91.798887,141.277751,53.076448,...,11.370323,17.725494,66.409818,54.254766,54.254766,-3.096053,11.370323,0.008419,49.256670,7.031358
3,MMM,2021-06-30,Q2,1.731250,1.111335,0.497352,203.543575,92.345836,142.534663,55.899555,...,12.894305,18.931108,77.296740,61.157410,61.157410,-2.944839,12.894305,0.007435,51.274895,7.987544
4,MMM,2021-03-31,Q1,1.834868,1.190243,0.554347,198.644786,88.667403,137.648309,53.104972,...,12.637073,17.218401,81.168894,66.262284,66.262284,1.033104,12.637073,0.007671,49.813703,8.128687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,MMM,1994-09-30,Q3,1.924565,0.973920,0.085250,285.311518,104.434706,174.785492,34.864154,...,6.094335,17.067713,80.001241,39.659899,39.659899,42.967707,6.094335,0.007947,30.794747,3.495550
111,MMM,1994-06-30,Q2,1.993552,1.012603,0.090563,287.974549,103.257677,174.265100,36.248331,...,5.564394,15.298028,135.412218,50.820566,50.820566,1.223842,5.564394,0.008862,27.620263,3.195629
112,MMM,1994-03-31,Q1,1.847309,0.967222,0.106528,280.885187,102.785978,172.045339,36.614391,...,5.815732,17.257140,109.444244,48.446649,48.446649,2.323719,5.815732,0.008900,30.161887,3.308700
113,MMM,1993-12-31,Q4,1.938757,0.995125,0.083486,313.635841,118.600439,195.935304,43.369923,...,6.663702,20.282350,125.421669,47.256637,47.256637,-2.151066,6.663702,0.007671,33.092153,3.563116
